In [ ]:
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
from git import Repo

# Repository URL
repo_url = "git@github.com:LeonVitanos/Wallch.git"

# Clone or fetch the repository
repo_path = "Downloads/wallch"  # Path where the repository will be cloned or fetched
if not os.path.exists(repo_path):
    Repo.clone_from(repo_url, repo_path)
else:
    repo = Repo(repo_path)
    repo.remote().fetch()

# Create the stats dictionary
languages = ['C++', 'Qt', 'C/C++ Header', 'QML']
stats = {}

# Loop through the commits and collect relevant data for first commit of each month
for commit in repo.iter_commits():
    date = datetime.fromtimestamp(commit.committed_date)
    year_month = date.strftime("%Y:%m")

    # Check if current commit is the first commit of the month
    if year_month not in stats:
        # Create year_month key if not exists
        stats[year_month] = {}

        # Run cloc to gather the stats for each language (first line is the header, and the rest are the data lines for each language)
        # the order of the cloc output data is: files,language,blank,comment,code
        cloc_output = os.popen("cloc --by-file --csv --quiet --include-lang=" + ",".join(languages) + " --git " + repo_path).read()
        lines = cloc_output.splitlines()

        # Extract the stats from the corresponding line in cloc output
        for line in lines:
            data = line.split(",")
            language = data[1]

            # Extract the stats using the known indices and store in stats dictionary
            if language in languages:
                stats[year_month][language] = {
                    'files': int(data[0]),
                    'blank': int(data[2]),
                    'comment': int(data[3]),
                    'code': int(data[4])
                }